In [6]:
#Necessary Imports
!pip install ta
import numpy as np
import pandas as pd
import os
import pickle
import ta
from statsmodels.tsa.arima_model import ARIMA
from google.colab import drive
drive.mount('/content/drive')
os.chdir("/content/drive/My Drive/mining-on-stock") #Data Source Location

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.10.2-py3-none-any.whl size=29103 sha256=206c58f7430ce36e771b0e5d4c94fa3ed544ce47df1319accb4ca8f9a063412d
  Stored in directory: /root/.cache/pip/wheels/47/51/06/380dc516ea78621870b93ff65527c251afdfdc5fa9d7f4d248
Successfully built ta
Mounted at /content/drive


In [ ]:
#Fetching training data from the pkl file.
with open('training_set.pkl', 'rb') as train_file:
  train_data = pickle.load(train_file)

#We can also use pd.read_pickle fuction

In [ ]:
# Concatenate the daily percentage changes for all the stocks into a single list
change_list = [df['Close'].pct_change() for df in train_data]
all_changes = sorted([item for sublist in change_list for item in sublist])
all_changes = np.nan_to_num(all_changes)

# Compute the two thresholds that divide the list into three equal-sized intervals
level1 = np.percentile(all_changes, 33.33)
level2 = np.percentile(all_changes, 66.67)

# Assign labels to the daily percentage changes for each stock
labels = []
for df in train_data:
    close_pct_change = df['Close'].pct_change() 
    label = pd.cut(close_pct_change, bins=[-np.inf, level1, level2, np.inf], labels=['decrease', 'no big change', 'increase'])
    labels.append(label)
    
# Output the thresholds and the number of data points in each level
print(f'Threshold 1: {level1:.6f}')
print(f'Threshold 2: {level2:.6f}')
labels = pd.concat(labels)
print(f'Total number of data points: {len(labels)}')
print(f'Number of data points in decrease level: {(labels == "decrease").sum()}')
print(f'Number of data points in no big change level: {(labels == "no big change").sum()}')
print(f'Number of data points in increase level: {(labels == "increase").sum()}')

Threshold 1: -0.002106
Threshold 2: 0.001981
Total number of data points: 4404000
Number of data points in decrease level: 1467857
Number of data points in no big change level: 1466290
Number of data points in increase level: 1467853


In [ ]:
#These are the functions that we have considered using for making additional features in our dataset. 
feature_funcs = [('awesome_oscillator', None),
 ('bollinger_percent_b', None),
 ('cfo', None),
 ('cmo', None),
 ('coppock_curve', None),
 ('detrended_price_oscillator', None),
 ('disparity_index', None),
 ('elder_impulse_system', None),
 ('gator_oscillator', None), 
 ('historical_volatility', None),
 ('intraday_momentum_index', None),
 ('linear_reg_slope', None),
 ('macd', None),
 ('macd_divergence', None), 
 ('momentum_indicator', None), 
 ('qstick', 14), 
 ('projected_volume_at_time', 14),
 ('projected_aggregate_volume', 14),
 ('price_volume_trend', None),
 ('positive_volume_index', None),
 ('twiggs_money_flow', None),
 ('volume_underlay', None),
 ('on_balance_volume', None),
 ('moving_average', 30),
 ('parabolic_sar', (0.02, 0.2)),
 ('pretty_good_oscillator', (10, 20)),
 ('price_momentum_oscillator', (12, 24)),
 ('price_oscillator', (12, 26, 9)),
 ('price_rate_of_change', 14),
 ('KST', (10, 15, 20, 30, 10, 10, 10, 15)),
 ('SpecialK', (10, 15, 20, 30, 10, 10, 10, 15)),
 ('ravi', (7, 65)),
 ('rsi', 14),
 ('rsi_divergence', (14, 9)),
 ('rainbow_moving_average', 30),
 ('rainbow_oscillator', (10, 1.0)),
 ('random_walk_index', 14),
 ('relative_vigor_index', 14),
 ('schaff_trend_cycle', 10),
 ('standard_deviation', 14),
 ('stochastic_divergence', 14),
 ('stochastic_momentum_index', (14, 3)),
 ('stochastic_rsi', (14, 3, 3)),
 ('stochastics', (14, 3, 3)),
 ('trix', 15),
 ('trade_volume_index', None),
 ('trend_intensity_index', 14),
 ('true_range', None),
 ('typical_price', None),
 ('ultimate_oscillator', (7, 14, 28, 4, 2, 1)),
 ('vwap', None),
 ('vertical_horizontal_filter', 28),
 ('vortex_indicator', 14),
 ('williams_r', 14),
 ('zigzag', 5),
 ('calculate_adx', None), 
 ('atr_bands', None),
 ('accumulative_swing_index', None),
 ('alligator', None),
 ('aroon', None),
 ('aroon_oscillator', None),
 ('average_true_range', None),
 ('cog', None),
 ('cpr', None),
 ('chv', None),
 ('cmf', None),
 ('ci', None),
 ('cci', None),
 ('donchian_channel', None),
 ('ehlers_fisher_transform', None),
 ('elder_ray_index', None),
 ('fractal_chaos_bands', None),
 ('fractal_chaos_oscillator', None),
 ('gopalakrishnan_range_index', None),
 ('high_minus_low', None),
 ('highest_high', None),
 ('ichimoku_clouds', None),
 ('keltner_channel', None),
 ('linear_reg_forecast', None),
 ('mass_index', None),
 ('median_price', None),
 ('money_flow_index', None)]
 #There are about 82 functions above from which we have obtained more than 100 new features.
 #With careful evaluation, we finally considered using 101 features for task-3 removing all other erratic features.
 #Basic Stats of the training data.
 


In [ ]:
len(feature_funcs)

82

In [ ]:
# Define a function to calculate the new features
from features import *
# print(dir(features))
def calculate_new_features(df):
    df_new = df.copy()
    for func in feature_funcs:
      func_name = func[0]
      func_args = func[1]
      # Import the function dynamically using its name
      func_module = __import__('features', globals(), locals(), [func_name], 0)
      func_to_call = getattr(func_module, func_name)
      # print(dir(func_module))
      # print(func_name, func_args)
      # Call the function with its arguments
      try:
        if func_args is None: 
            result = func_to_call(df)
        elif isinstance(func_args, tuple):
            result = func_to_call(df, *func_args)
        else:
            result = func_to_call(df, func_args)   
        if isinstance(result, pd.DataFrame):
          for i in result.columns.tolist():
            column_name = f'{func_name} ({i})'
            df_new[column_name] = result[i]
        elif isinstance(result, pd.Series):
            column_name = f'{func_name} ({result.name})'
            df_new[column_name] = result
      except AttributeError:
          # handle the exception
          print(f"{func_name} function not found in the features module")
      except Exception as e:
          # handle any other exception
          print(f"An error occurred while calling {func_name}: {str(e)}")
    return df_new

# Create a new list to store the processed dataframes
processed_data = []

# Iterate over each dataframe in the list
for df in train_data.copy():
    # Apply the calculate_new_features function to the current dataframe
    df_new = calculate_new_features(df)
    # Append the updated dataframe to the new list
    processed_data.append(df_new)
#print(processed_data)

In aroon_oscillator function


<ipython-input-8-518118f18b88>:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_new[column_name] = result[i]
<ipython-input-8-518118f18b88>:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_new[column_name] = result
<ipython-input-8-518118f18b88>:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

In aroon_oscillator function


<ipython-input-8-518118f18b88>:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_new[column_name] = result[i]
<ipython-input-8-518118f18b88>:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_new[column_name] = result
<ipython-input-8-518118f18b88>:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

In aroon_oscillator function


...

<ipython-input-8-518118f18b88>:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_new[column_name] = result[i]
<ipython-input-8-518118f18b88>:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_new[column_name] = result
<ipython-input-8-518118f18b88>:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

In aroon_oscillator function


<ipython-input-8-518118f18b88>:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_new[column_name] = result[i]
<ipython-input-8-518118f18b88>:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_new[column_name] = result
<ipython-input-8-518118f18b88>:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

In aroon_oscillator function


<ipython-input-8-518118f18b88>:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_new[column_name] = result[i]
<ipython-input-8-518118f18b88>:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_new[column_name] = result
<ipython-input-8-518118f18b88>:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

In [9]:
from features import *
def feature_engineering_func():
  #These are the functions that we have considered using for making additional features in our dataset. 
  feature_funcs = [('awesome_oscillator', None),
  ('bollinger_percent_b', None),
  ('cfo', None),
  ('cmo', None),
  ('coppock_curve', None),
  ('detrended_price_oscillator', None),
  ('disparity_index', None),
  ('elder_impulse_system', None),
  ('gator_oscillator', None), 
  ('historical_volatility', None),
  ('intraday_momentum_index', None),
  ('linear_reg_slope', None),
  ('macd', None),
  ('macd_divergence', None), 
  ('momentum_indicator', None), 
  ('qstick', 14), 
  ('projected_volume_at_time', 14),
  ('projected_aggregate_volume', 14),
  ('price_volume_trend', None),
  ('positive_volume_index', None),
  ('twiggs_money_flow', None),
  ('volume_underlay', None),
  ('on_balance_volume', None),
  ('moving_average', 30),
  ('parabolic_sar', (0.02, 0.2)),
  ('pretty_good_oscillator', (10, 20)),
  ('price_momentum_oscillator', (12, 24)),
  ('price_oscillator', (12, 26, 9)),
  ('price_rate_of_change', 14),
  ('KST', (10, 15, 20, 30, 10, 10, 10, 15)),
  ('SpecialK', (10, 15, 20, 30, 10, 10, 10, 15)),
  ('ravi', (7, 65)),
  ('rsi', 14),
  ('rsi_divergence', (14, 9)),
  ('rainbow_moving_average', 30),
  ('rainbow_oscillator', (10, 1.0)),
  ('random_walk_index', 14),
  ('relative_vigor_index', 14),
  ('schaff_trend_cycle', 10),
  ('standard_deviation', 14),
  ('stochastic_divergence', 14),
  ('stochastic_momentum_index', (14, 3)),
  ('stochastic_rsi', (14, 3, 3)),
  ('stochastics', (14, 3, 3)),
  ('trix', 15),
  ('trade_volume_index', None),
  ('trend_intensity_index', 14),
  ('true_range', None),
  ('typical_price', None),
  ('ultimate_oscillator', (7, 14, 28, 4, 2, 1)),
  ('vwap', None),
  ('vertical_horizontal_filter', 28),
  ('vortex_indicator', 14),
  ('williams_r', 14),
  ('zigzag', 5),
  ('calculate_adx', None), 
  ('atr_bands', None),
  ('accumulative_swing_index', None),
  ('alligator', None),
  ('aroon', None),
  ('aroon_oscillator', None),
  ('average_true_range', None),
  ('cog', None),
  ('cpr', None),
  ('chv', None),
  ('cmf', None),
  ('ci', None),
  ('cci', None),
  ('donchian_channel', None),
  ('ehlers_fisher_transform', None),
  ('elder_ray_index', None),
  ('fractal_chaos_bands', None),
  ('fractal_chaos_oscillator', None),
  ('gopalakrishnan_range_index', None),
  ('high_minus_low', None),
  ('highest_high', None),
  ('ichimoku_clouds', None),
  ('keltner_channel', None),
  ('linear_reg_forecast', None),
  ('mass_index', None),
  ('median_price', None),
  ('money_flow_index', None)]
  #There are about 82 functions above from which we have obtained more than 100 new features.
  #With careful evaluation, we finally considered using 101 features for task-3 removing all other erratic features.
  # Concatenate the daily percentage changes for all the stocks into a single list
  change_list = [df['Close'].pct_change() for df in train_data]
  all_changes = sorted([item for sublist in change_list for item in sublist])
  all_changes = np.nan_to_num(all_changes)

  # Compute the two thresholds that divide the list into three equal-sized intervals
  level1 = np.percentile(all_changes, 33.33)
  level2 = np.percentile(all_changes, 66.67)

  # Assign labels to the daily percentage changes for each stock
  labels = []
  for df in train_data:
      close_pct_change = df['Close'].pct_change() 
      label = pd.cut(close_pct_change, bins=[-np.inf, level1, level2, np.inf], labels=['decrease', 'no big change', 'increase'])
      labels.append(label)
      
  # Output the thresholds and the number of data points in each level
  labels = pd.concat(labels)
  # Define a function to calculate the new features
  # print(dir(features))
  def calculate_new_features(df):
      df_new = df.copy()
      for func in feature_funcs:
        func_name = func[0]
        func_args = func[1]
        # Import the function dynamically using its name
        func_module = __import__('features', globals(), locals(), [func_name], 0)
        func_to_call = getattr(func_module, func_name)
        # print(dir(func_module))
        # print(func_name, func_args)
        # Call the function with its arguments
        try:
          if func_args is None: 
              result = func_to_call(df)
          elif isinstance(func_args, tuple):
              result = func_to_call(df, *func_args)
          else:
              result = func_to_call(df, func_args)   
          if isinstance(result, pd.DataFrame):
            for i in result.columns.tolist():
              column_name = f'{func_name} ({i})'
              df_new[column_name] = result[i]
          elif isinstance(result, pd.Series):
              column_name = f'{func_name} ({result.name})'
              df_new[column_name] = result
        except AttributeError:
            # handle the exception
            print(f"{func_name} function not found in the features module")
        except Exception as e:
            # handle any other exception
            print(f"An error occurred while calling {func_name}: {str(e)}")
      return df_new

  # Create a new list to store the processed dataframes
  processed_data = []

  # Iterate over each dataframe in the list
  for df in train_data.copy():
      # Apply the calculate_new_features function to the current dataframe
      df_new = calculate_new_features(df)
      # Append the updated dataframe to the new list
      processed_data.append(df_new)
  #print(processed_data)
  #Now checking if training data has any infinity values in them 
  list_inf = []
  for i in range(0, len(train_data)):
      inf_locs = np.where(np.isinf(train_data[i].iloc[:, :-1]))
      list_inf.append((i, inf_locs))  
  #Dropping rows that contain nan's in the label column for all the data frames in the train data.
  #Those are actually the first rows of each dataframe for which the nan's in the labels caused because of percent change.  
  i = 0
  while(i < len(train_data)):
    train_data[i] = train_data[i].dropna(subset = ['Label'])
    i = i +1
  #Now each dataframe has 2201 rows.
  #Droping 16th column as we dropeed in training
  #Droping 16th column as we dropeed in training
  i = 0
  while(i < len(train_data)):
    train_data[i] = train_data[i].drop(train_data[i].columns[16], axis =1)
    i = i +1
  list_inf = []
  for i in range(0, len(train_data)):
      inf_locs = np.where(np.isinf(train_data[i].iloc[:, :-1]))
      list_inf.append((i, inf_locs))
  #16th column is the culprit having inf values
  for i, j in list_inf:
    if len(j[0])!=0:
      k = 0
      while(k < len(train_data)):
        # select column containing infinite values
        col = train_data[k].iloc[:, j[1]]
      # replace infinite values with NaN
        col.replace([np.inf, -np.inf], np.nan, inplace=True)
      # replace NaN values with maximum value of column
        max_val = col.max()
        col.fillna(max_val, inplace=True)
        k = k + 1
        #Removing the above obtained features
  for i in range(0, len(train_data)):
    for k in ['coppock_curve (coppock)','twiggs_money_flow (Twiggs Money Flow)' ,'volume_underlay (Volume Ratio MA)', 'stochastic_rsi (Close)', 'vwap (None)', 'donchian_channel (upper_dc)', 'donchian_channel (lower_dc)', 'ichimoku_clouds (senkou_span_b)']:
      train_data[i] = train_data[i].drop(k, axis=1)
  #10
  for i in range(0, len(train_data)):
    for j in ['cfo (cfo)', 'ichimoku_clouds (chikou_span)', 'cci (cci)', 'chv (chv)', 'cmf (cmf)', 'money_flow_index (Money Flow Index)', 'relative_vigor_index (None)']:
      train_data[i][j].fillna(method='bfill', inplace=True)
    for j in ['KST (Close)', 'SpecialK (Close)', 'calculate_adx (ADX)']:
      train_data[i][j].fillna(method='ffill', inplace=True)
#14
  for i in range(0, len(train_data)):
    for j in range(0, train_data[0].shape[1]):
      train_data[i].iloc[:, j].fillna(method='bfill', inplace=True)
      train_data[i].iloc[:, j].fillna(method='ffill', inplace=True)
      train_data = pd.concat(train_data, axis =0)
  return train_data

In [ ]:
#Saving the training data with newly obtained features:
with open('data_with_new_features.pkl', 'wb') as f:
    pickle.dump(processed_data, f)

In [ ]:
#Loading the training data with newly obtained features:
with open('data_with_new_features.pkl', 'rb') as train_file_1:
    train_data = pickle.load(processed_data, train_file_1)

In [ ]:
#Now checking if training data has any infinity values in them 
list_inf = []
for i in range(0, len(train_data)):
    inf_locs = np.where(np.isinf(train_data[i].iloc[:, :-1]))
    list_inf.append((i, inf_locs))

In [ ]:
print(list_inf[:][len(list_inf[:][1])!=0])

In [ ]:
#Dropping rows that contain nan's in the label column for all the data frames in the train data.
#Those are actually the first rows of each dataframe for which the nan's in the labels caused because of percent change.  
i = 0
while(i < len(train_data)):
  train_data[i] = train_data[i].dropna(subset = ['Label'])
  i = i +1
#Now each dataframe has 2201 rows.

In [ ]:
#3
i = 0
lis = [0]*train_data[0].shape[1]
for df in train_data:
  lis1 = []
  for i in range(0, df.shape[1]):
    lis1.append(df.iloc[:, i].isna().sum())
  for j in range(0, df.shape[1]):
    lis[j] = lis[j] + lis1[j]

In [ ]:
#Checking the features that has NAN count greater than 2.5% of the data.
columns = []
count = 0
for i in lis:
  if(i > (2201*2000)*2.5/100):
    print(train_data[0].columns[count], i)
    columns.append(train_data[0].columns[count])
  count = count +1

coppock_curve (coppock) 136000
twiggs_money_flow (Twiggs Money Flow) 358426
volume_underlay (Volume Ratio MA) 362134
stochastic_rsi (Close) 905784
vwap (None) 321387
donchian_channel (upper_dc) 294000
donchian_channel (lower_dc) 294000
ichimoku_clouds (senkou_span_b) 152000


In [ ]:
#Removing the above obtained features
for j in columns:
  for i in range(0, len(train_data)):
    train_data[i] = train_data[i].drop([j], axis=1)

In [ ]:
lissy = []
for i in range(0, len(train_data)):
    inf_locs = np.where(np.isinf(train_data[i].iloc[:, :-1]))
    lissy.append((i, inf_locs))